# Train OCR text Detector quick example

For train datasets please download last version of ocr datasets [https://nomeroff.net.ua/datasets/](https://nomeroff.net.ua/datasets/). Unpack archive and rename to **./datasets/ocr** .
For examle
```bash
cd ./datasets/ocr
wget https://nomeroff.net.ua/datasets/autoriaNumberplateOcrUa-1995-2019-07-30.zip
unzip autoriaNumberplateOcrUa-1995-2019-07-30.zip
mv autoriaNumberplateOcrUa-1995-2019-07-30 ua
```
or use your own dataset.

In [4]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

DATASET_NAME = "ua-1995"
VERSION = "2"
MODE = "cpu"
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "datasets/ocr/", DATASET_NAME)
RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.h5'.format(DATASET_NAME, VERSION, MODE))

FROZEN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/", 'anpr_ocr_{}_{}-{}.pb'.format(DATASET_NAME, VERSION, MODE))

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.Base import OCR, convert_keras_to_freeze_pb

In [5]:
class eu_ua_1995(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        self.letters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'E', 'H', 'I', 'K', 'M', 'O', 'P', 'T', 'X']
        
        self.EPOCHS = 1

In [6]:
ocrTextDetector = eu_ua_1995()
model = ocrTextDetector.prepare(PATH_TO_DATASET, aug_count=0)

GET ALPHABET
Max plate length in "val": 7
Max plate length in "train": 7
Max plate length in "test": 7
Letters train  {'5', 'E', 'O', '8', 'P', '4', '6', '2', '3', '1', 'B', 'K', 'I', 'C', 'M', '9', '7', 'X', 'T', '0', 'A', 'H'}
Letters val  {'5', 'E', 'O', '8', 'P', '4', '6', '2', '3', '1', 'B', 'K', 'I', 'C', 'M', '9', '7', 'X', 'T', '0', 'A', 'H'}
Letters test  {'5', 'E', 'O', '8', 'P', '4', '6', '2', '3', '1', 'B', 'K', 'I', 'C', 'M', '9', '7', 'X', 'T', '0', 'A', 'H'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H I K M O P T X

EXPLAIN DATA TRANSFORMATIONS
Text generator output (data which will be fed into the neutral network):
1) the_input (image)
2) the_labels (plate number): 02953MA is encoded as [0, 2, 9, 5, 3, 17, 10]
3) input_length (width of image that is fed to the loss function): 30 == 128 / 4 - 2
4) label_length (length of plate number): 7
START BUILD DATA
DATA PREPARED


In [7]:
# model = ocrTextDetector.load(RESULT_MODEL_PATH)
# ocrTextDetector.test(verbose=True)

In [8]:
# A = {"3", "S", "P", "0", "H", "7", "C", "6", "I", "X", "1", "E", "2", "5", "8", "D", "4", "O", "K", "B", "A", "9", "M", "T"}
# B = {'3', '0', 'H', '7', 'C', '6', 'I', 'X', '1', 'E', '2', '5', '8', '9', '4', 'O', 'K', 'B', 'A', 'P', 'M', 'T'}
# A - B

In [12]:
model = ocrTextDetector.train(mode=MODE)


START TRAINING
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_eu_ua_1995 (InputLaye (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_eu_ua_1995[0][0]       
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_____________________________________________________________________________________________

In [13]:
ocrTextDetector.test(verbose=True)


RUN TEST

Predicted: 		 43498CK
True: 			 43498OK

Predicted: 		 21099KH
True: 			 21099AH

Predicted: 		 46013AE
True: 			 43013AE

Predicted: 		 89932MO
True: 			 88932MO

Predicted: 		 88873IX
True: 			 B8873IX

Predicted: 		 23433XM
True: 			 22333XM

Predicted: 		 6397HA
True: 			 6397BHA

Predicted: 		 83296EE
True: 			 83296EA

Predicted: 		 61399PEH
True: 			 61399PE

Predicted: 		 04653OI
True: 			 04653OT

Predicted: 		 68867HX
True: 			 68867MO

Predicted: 		 56967KX
True: 			 56567KX

Predicted: 		 51560KK
True: 			 51560XK

Predicted: 		 34843TC
True: 			 34643TC

Predicted: 		 65281PE
True: 			 65261PE

Predicted: 		 09907OH
True: 			 09907CH

Predicted: 		 76685MHA
True: 			 76685HA
acc: 0.9838249286393911


In [14]:
ocrTextDetector.save(RESULT_MODEL_PATH, verbose=True)
#model = ocrTextDetector.load(RESULT_MODEL_PATH)

SAVED TO /mnt/data/var/www/html2/js/nomeroff-net_2/models/anpr_ocr_ua-1995_2-cpu.h5


In [15]:
model = ocrTextDetector.load(RESULT_MODEL_PATH)

### Convert keras OCR  .h5 model to .pb graph

In [16]:
import keras
keras.backend.clear_session()
model = ocrTextDetector.load(RESULT_MODEL_PATH)
convert_keras_to_freeze_pb(model, FROZEN_MODEL_PATH)

OUTPUT: softmax_eu_ua_1995/truediv
INPUT: the_input_eu_ua_1995
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_eu_ua_1995 (InputLaye (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_eu_ua_1995[0][0]       
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
______________________________________________

W0731 18:35:06.188419 139689264326272 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0731 18:35:06.378947 139689264326272 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/tools/freeze_graph.py:233: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0731 18:35:06.379428 139689264326272 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a fut